<a href="https://colab.research.google.com/github/ykitaguchi77/manipulate_CSV/blob/master/DiseaseName_extraction1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**病名を含むリストを抽出するスクリプト**
csvから特定の文字列を含む行を抜き出して保存

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 使用方法の概要

   - `byoumei`リストに抽出したい疾病名を追加。
   - 必要に応じて`age_range`を設定。


このスクリプトを活用することで、特定の疾病に関連するデータを効率的に抽出・整理し、分析やレポート作成に役立てることができます。


In [5]:
import csv
import numpy as np
import os
import os.path
import sys
import glob
import pandas as pd
import codecs

age_range = []  # Age range as a list ex, [0,10]
byoumei=['眼窩脂肪ヘルニア']
csv_path = '/content/drive/MyDrive/Deep_learning/●DiseaseInfo_modified1.5_connected_9800.csv'
dst_folder = '/content/'

#各ファイルを開く　※codec errorを回避
#with codecs.open(csv_path, "r", "utf-8", "ignore") as file:
with codecs.open(csv_path, "r", "shift_jis", "ignore") as file:
    df = pd.read_csv(file, delimiter=",", header=None)
#print(df)

#ファイル名設定のための前処理
byoumei_filename = byoumei[0]
j=1
for i in range(len(byoumei)-1):
    byoumei_filename = byoumei_filename + '.' + byoumei[j]
    j+=1

# 年齢の制約があればファイル名に含める
if age_range:
    age_range_str = f"{age_range[0]}-{age_range[1]}"
    dst_path = dst_folder + "/Disease_extracted_" + byoumei_filename + f"_age_{age_range_str}.csv"
else:
    dst_path = dst_folder + "/Disease_extracted_" + byoumei_filename + ".csv"


result_list=pd.DataFrame({}) #空のリストを定義

for i in byoumei:
    for j in range(df.shape[1]):  #それぞれの列に対して検索をかける
        extract = df[df[j].astype(str).str.contains(i, na=False)] #数字、naに関するエラーを回避
        result_list = pd.concat([result_list, extract])


#重複した行を削除
result_list = result_list[~result_list.duplicated()]

# Type casting first
result_list.loc[:, 0] = result_list.loc[:, 0].astype('int')
result_list.loc[:, 2] = result_list.loc[:, 2].astype('int')

# Filter and sort by age, then by patient number
if age_range:  # Check if age_range is not empty
    result_list = result_list[
        (result_list[4] >= age_range[0]) & (result_list[4] <= age_range[1])
    ].sort_values(by=[2], ascending=True).reset_index(drop=True)
else:  # If age_range is empty, skip age filtering
    result_list = result_list.sort_values(by=[2], ascending=True).reset_index(drop=True)

print(str(len(result_list))+"例抽出されました！")

#result_list

### 抽出したファイルを出力
result_list.to_csv(dst_path, encoding='utf_8_sig', index = False, header=False)

<ipython-input-5-fb4b309aeb38>:18: DtypeWarning: Columns (2,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,43,45,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,88,89,101,103,104,105,106,107,108,109,110,117,119,120,124,131,133,134,135,136,137) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, delimiter=",", header=None)


21例抽出されました！
